# Peewee

**[Peewee](https://docs.peewee-orm.com/en/latest/index.html)** é um [ORM](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping) simples e pequeno. Possui poucos (mas expressivos) conceitos, tornando-o fácil de aprender e intuitivo de usar.

Suporta sqlite, mysql, postgresql e cockroachdb.

O código-fonte de Peewee hospedado no [GitHub](https://github.com/coleifer/peewee).

Para instalar peewee é muito simples, podemos usar o [pip](https://pypi.org/project/pip/):

```
pip install peewee
```

## Tutorial

Este documento apresenta uma visão geral breve e de alto nível dos principais recursos do Peewee. Este guia cobrirá:

- Definição de Modelos
- Armazenamento de Dados
- Recuperação de Dados

Para maiores detalhes, lembre de consultar a [documentação oficial](https://docs.peewee-orm.com/en/latest/index.html).

### Definição de Modelo

Classes de modelo, campos e instâncias de modelo mapeiam para conceitos de banco de dados:

| Objeto  | Correspondente |
|---|---|
| Classes de Modelo  | Tabela de Banco de Dados  |
| Instância de Campo  | Coluna de uma Tabela |
| Instância de Modelo  | Linha de uma Tabela  |

Ao iniciar um projeto com peewee, normalmente é melhor começar com seu modelo de dados, definindo uma ou mais classes de modelo (`Model`):

In [25]:
from peewee import *

db = SqliteDatabase('livros.db')

class Autor(Model):
    nome = CharField()
    nacionalidade = CharField()
    nascimento = DateField()

    class Meta:
        database = db # Esse modelo usa o banco de dados "livros.db"

**Observação**: O Peewee inferirá automaticamente o nome da tabela do banco de dados a partir do nome da classe. Você pode substituir o nome padrão especificando um atributo **table_name** na classe interna “Meta” (junto com o atributo **database**). Para aprender mais sobre como o Peewee gera nomes de tabelas, consulte a seção [Nomes de Tabelas](https://docs.peewee-orm.com/en/latest/peewee/models.html#table-names) da documentação.

Perceba também que chamamos nosso modelo de **Autor** em vez de **Autores**. Esta é uma convenção que você deve seguir - embora a tabela contenha vários autores, sempre nomeamos a classe usando a forma singular.

Existem muitos [tipos de campo](https://docs.peewee-orm.com/en/latest/peewee/models.html#fields) adequados para armazenar vários tipos de dados. O Peewee lida com a conversão entre os valores pythônicos e aqueles usados pelo banco de dados, então você pode usar os tipos Python em seu código sem se preocupar.

As coisas ficam interessantes quando configuramos relacionamentos entre modelos usando [relacionamentos de chave estrangeira](https://docs.peewee-orm.com/en/latest/peewee/relationships.html#relationships). Isso é simples com peewee:

In [26]:
class Livro(Model):
    autor = ForeignKeyField(Autor, backref='livros')
    titulo = CharField()
    genero = CharField()

    class Meta:
        database = db # Esse modelo usa o banco de dados "livros.db"

Agora que temos nossos modelos, vamos nos conectar ao banco de dados. Embora não seja necessário abrir a conexão explicitamente, é uma boa prática, pois isso revelará quaisquer erros com sua conexão de banco de dados imediatamente, ao contrário de algum tempo arbitrário depois, quando a primeira consulta for executada. Também é bom fechar a conexão quando terminar - por exemplo, um aplicativo da web pode abrir uma conexão ao receber uma solicitação e fechar a conexão ao enviar a resposta.

In [27]:
db.connect()

True

Começaremos criando as tabelas no banco de dados que armazenarão nossos dados. 

O comando a seguir criará as tabelas com as colunas, índices, sequências e restrições de chave estrangeira apropriadas:

In [28]:
db.create_tables([Autor, Livro])

### Armazenando Dados

Vamos começar preenchendo o banco de dados com alguns **autores**. Usaremos os métodos **save()** e **create()** para adicionar e atualizar os registros das autores.

In [29]:
from datetime import date

huxley = Autor(nome="Aldous Leonard Huxley", nacionalidade="Inglês", nascimento=date(1894,11,22))
huxley.save()

1

Huxley agora está armazenado no banco de dados

**Observação**: Quando você chama o método **save()**, o número de linhas modificadas é retornado.

Você também pode adicionar um autor chamando o método **create()**, que retorna uma instância de modelo:

In [30]:
george = Autor.create(nome='George', nacionalidade="Inglês", nascimento=date(1903,1,21))
machado = Autor.create(nome='Machado de Assis', nacionalidade="Brasileiro", nascimento=date(1839,6,21))

Para atualizar uma linha, modifique a instância do modelo e chame **save()** para persistir as mudanças. Aqui, vamos mudar o nome de **george** e, em seguida, salvar as alterações no banco de dados:

In [31]:
george.nome = "George Orwell"
george.save()

1

Agora armazenamos 3 autores no banco de dados. Vamos cadastrar alguns de seus livros:

In [32]:
orwell_livro = Livro.create(autor=george, titulo="Nineteen Eighty-Four", genero="dystopian social science fiction")
huxley_livro = Livro.create(autor=huxley, titulo="Brave New World", genero="dystopian social science fiction")
machado_livro = Livro.create(autor=machado, titulo="Memórias Póstumas de Brás Cubas", genero="Novel")

Se eventualmente quisermos deletar uma instância, podemos usar **delete_instance()**, que irá retornar o número de linhas removidas do banco de dados.

In [33]:
orwell_livro.delete_instance()

1

#### Inserindo Múltiplos Valores

Existem algumas maneiras de carregar muitos dados rapidamente. A abordagem ingênua é simplesmente chamar **Model.create()** em um loop:

In [47]:
autores = [
    {'nome': 'William Gibson', 'nacionalidade': 'Canadense', 'nascimento': date(1948,3,17)},
    {'nome': 'Monteiro Lobato', 'nacionalidade': 'Brasileiro', 'nascimento': date(1882,7,4)},
    {'nome': 'Gabriel García Márquez', 'nacionalidade': 'Colombiano', 'nascimento': date(1927,3,6)}
]

for autor in autores:
    Autor.create(**autor)

A abordagem acima é lenta por alguns motivos:

1. Se você não estiver envolvendo o loop em uma transação, então cada chamada para **create()** acontecerá em sua própria transação. Isso vai ser muito lento!
2. Há uma quantidade razoável de lógica Python atrapalhando seu caminho, e cada **InsertQuery** deve ser gerado e *parsed* em SQL.
3. São muitos dados (em termos de bytes brutos de SQL) que você está enviando para seu banco de dados fazer *parse*.

Você pode obter uma aceleração significativa simplesmente envolvendo isso em uma transação com **atomic()**, que é muito mais rápido:

In [55]:
novos_autores = [
    {'nome': 'Philip K. Dick', 'nacionalidade': 'Estado Unidense', 'nascimento': date(1928,12,16)},
    {'nome': 'J. R. R. Tolkien', 'nacionalidade': 'Inglês', 'nascimento': date(1892,1,3)},
    {'nome': 'Clarice Lispector', 'nacionalidade': 'Brasileira', 'nascimento': date(1920,12,10)}
]

with db.atomic():
    for novo_autor in novos_autores:
        Autor.create(**novo_autor)

Podemos obter outro grande impulso usando **insert_many()**. Este método aceita uma lista de tuplas ou dicionários e insere várias linhas em uma única consulta:

In [66]:
philip = Autor.get(Autor.nome == 'Philip K. Dick')
tolkien = Autor.get(Autor.nome == 'J. R. R. Tolkien')
clarice = Autor.get(Autor.nome == 'Clarice Lispector')

livros = [
    {'autor': philip, 'titulo': 'The Cosmic Puppets', 'genero': 'Science Fiction'},
    {'autor': tolkien, 'titulo': 'The Lord of the Rings', 'genero': 'Epic Fantasy'},
    {'autor': clarice, 'titulo': 'The Hour of the Star', 'genero': 'Novel'}
]

Livro.insert_many(livros).execute()

3

O método **insert_many()** também aceita uma lista de tuplas de linha, desde que você também especifique os campos correspondentes:

In [67]:
dados = [
    (philip, 'The World She Wanted', 'Science Fiction'),
    (tolkien, 'The Hobbit', 'Epic Fantasy'),
    (huxley, 'The Doors of Perception', 'Psychedelic'),
    (machado, 'Quincas Borba', 'Novel')
]

Livro.insert_many(dados, fields=[Livro.autor, Livro.titulo, Livro.genero]).execute()

4

Também é uma boa prática envolver a [inserção em massa](http://docs.peewee-orm.com/en/latest/peewee/querying.html#bulk-inserts) em uma transação:

In [68]:
novos_dados = [
    (philip, 'Do Androids Dream of Electric Sheep?', 'Science Fiction'),
    (huxley, 'Heaven and Hell', 'Philosophical'),
    (machado, 'Dom Casmurro', 'Novel')
]

campos = [Livro.autor, Livro.titulo, Livro.genero]

with db.atomic():
    Livro.insert_many(novos_dados, fields=campos).execute()

### Recuperando Dados

A verdadeira força de nosso banco de dados está em como ele nos permite recuperar dados por meio de consultas (*queries*). Bancos de dados relacionais são excelentes para fazer consultas ad-hoc.

#### Obtendo Registros Únicos

Vamos obter o registro de Huxley do banco de dados.

Para obter um único registro do banco de dados, use **select.get()**:

In [69]:
aldous = Autor.select().where(Autor.nome == 'Aldous Leonard Huxley').get()
aldous.nome

'Aldous Leonard Huxley'

Também podemos usar a abreviação equivalente **Model.get()**:

In [70]:
assis = Autor.get(Autor.nome == 'Machado de Assis')
assis.nome

'Machado de Assis'

#### Lista de Registros

Vamos listar todos os autores no banco de dados:

In [71]:
for autor in Autor.select():
    print(autor.nome)

Aldous Leonard Huxley
George Orwell
Machado de Assis
William Gibson
Monteiro Lobato
Gabriel García Márquez
Philip K. Dick
J. R. R. Tolkien
Clarice Lispector


Vamos listar todos os livros e seus autores:

In [72]:
query = Livro.select()
for livro in query:
    print(f'Livro Título: {livro.titulo} | Autor: {livro.autor.nome}')

Livro Título: Brave New World | Autor: Aldous Leonard Huxley
Livro Título: Memórias Póstumas de Brás Cubas | Autor: Machado de Assis
Livro Título: The Cosmic Puppets | Autor: Philip K. Dick
Livro Título: The Lord of the Rings | Autor: J. R. R. Tolkien
Livro Título: The Hour of the Star | Autor: Clarice Lispector
Livro Título: The World She Wanted | Autor: Philip K. Dick
Livro Título: The Hobbit | Autor: J. R. R. Tolkien
Livro Título: The Doors of Perception | Autor: Aldous Leonard Huxley
Livro Título: Quincas Borba | Autor: Machado de Assis
Livro Título: Do Androids Dream of Electric Sheep? | Autor: Philip K. Dick
Livro Título: Heaven and Hell | Autor: Aldous Leonard Huxley
Livro Título: Dom Casmurro | Autor: Machado de Assis


**Atenção**: Há um grande problema com a consulta anterior: como estamos acessando **livro.autor.nome** e não selecionamos essa relação em nossa consulta original, o peewee terá que realizar uma consulta adicional para recuperar o autor do livro. Esse comportamento é conhecido como [N + 1](https://docs.peewee-orm.com/en/latest/peewee/relationships.html#nplusone) e geralmente deve ser evitado.

Para obter um guia detalhado sobre como trabalhar com relacionamentos e **joins**, consulte a documentação [Relacionamentos e Joins](https://docs.peewee-orm.com/en/latest/peewee/relationships.html#relationships).

Podemos evitar as consultas extras selecionando Livro e Autor e adicionando um join:

In [78]:
query = (Livro
         .select(Livro, Autor)
         .join(Autor)
         .where(Livro.titulo == 'Brave New World'))

for livro in query:
    print(f'Título do Livro: {livro.titulo}, Autor: {livro.autor.nome}')

Título do Livro: Brave New World, Autor: Aldous Leonard Huxley


Vamos obter todos os livros escritos por Machado de Assis:

In [74]:
for livro in Livro.select().join(Autor).where(Autor.nome == 'Machado de Assis'):
    print(livro.titulo)

Memórias Póstumas de Brás Cubas
Quincas Borba
Dom Casmurro


Também podemos obter os livros de Machado de Assis de outra maneira. Uma vez que já temos um objeto representando Machado de Assis, podemos fazer o seguinte:

In [75]:
for livro in Livro.select().where(Livro.autor == machado):
    print(livro.titulo)

Memórias Póstumas de Brás Cubas
Quincas Borba
Dom Casmurro


#### Ordenando

Vamos garantir que nossos livros sejam ordenados em ordem alfabética adicionando uma cláusula **order_by()**:

In [79]:
for livro in Livro.select().order_by(Livro.titulo):
    print(livro.titulo)

Brave New World
Do Androids Dream of Electric Sheep?
Dom Casmurro
Heaven and Hell
Memórias Póstumas de Brás Cubas
Quincas Borba
The Cosmic Puppets
The Doors of Perception
The Hobbit
The Hour of the Star
The Lord of the Rings
The World She Wanted


Vamos listar todas os autores agora, do mais jovem ao mais velho:

In [80]:
for autor in Autor.select().order_by(Autor.nascimento.desc()):
    print(autor.nome, autor.nascimento)

William Gibson 1948-03-17
Philip K. Dick 1928-12-16
Gabriel García Márquez 1927-03-06
Clarice Lispector 1920-12-10
George Orwell 1903-01-21
Aldous Leonard Huxley 1894-11-22
J. R. R. Tolkien 1892-01-03
Monteiro Lobato 1882-07-04
Machado de Assis 1839-06-21


#### Combinação de Expressões de Filtro

Peewee suporta expressões aninhadas arbitrariamente. Vamos pegar todas as pessoas cujo aniversário foi:

- Antes de 1900
- Depois de 1940

In [81]:
d1900 = date(1900, 1, 1)
d1940 = date(1940, 1, 1)

query = (Autor
         .select()
         .where((Autor.nascimento < d1900) | (Autor.nascimento > d1940)))

for autor in query:
    print(autor.nome, autor.nascimento)

Aldous Leonard Huxley 1894-11-22
Machado de Assis 1839-06-21
William Gibson 1948-03-17
Monteiro Lobato 1882-07-04
J. R. R. Tolkien 1892-01-03


Agora vamos fazer o oposto. Buscar os autores cujo aniversário é entre 1900 e 1940:

In [83]:
query = (Autor
         .select()
         .where(Autor.nascimento.between(d1900, d1940)))

for autor in query:
    print(autor.nome, autor.nascimento)

George Orwell 1903-01-21
Gabriel García Márquez 1927-03-06
Philip K. Dick 1928-12-16
Clarice Lispector 1920-12-10


#### Agregados e Pré-busca

Agora vamos listar todas os autores e quantos livros eles têm:

In [84]:
for autor in Autor.select():
    print(autor.nome, autor.livros.count(), 'livros')

Aldous Leonard Huxley 3 livros
George Orwell 0 livros
Machado de Assis 3 livros
William Gibson 0 livros
Monteiro Lobato 0 livros
Gabriel García Márquez 0 livros
Philip K. Dick 3 livros
J. R. R. Tolkien 2 livros
Clarice Lispector 1 livros


Como podemos ver, alguns autores ainda não possuem livros cadastros e outros estão incompletos, mas já podemos ter uma ideia de como funciona.

Mais uma vez, encontramos um exemplo clássico de comportamento de consulta [N + 1](https://docs.peewee-orm.com/en/latest/peewee/relationships.html#nplusone). Neste caso, estamos executando uma consulta adicional para cada **Autor** retornada pelo **SELECT**! Podemos evitar isso executando um **JOIN** e usando uma função SQL para agregar os resultados.

In [85]:
query = (Autor
         .select(Autor, fn.COUNT(Livro.id).alias('livro_count'))
         .join(Livro, JOIN.LEFT_OUTER)  
         .group_by(Autor)
         .order_by(Autor.nome))

for autor in query:
    print(autor.nome, autor.livro_count, 'livros')

Aldous Leonard Huxley 3 livros
Clarice Lispector 1 livros
Gabriel García Márquez 0 livros
George Orwell 0 livros
J. R. R. Tolkien 2 livros
Machado de Assis 3 livros
Monteiro Lobato 0 livros
Philip K. Dick 3 livros
William Gibson 0 livros


**Observação**: O Peewee fornece um ajudante mágico **fn()**, que pode ser usado para chamar qualquer função SQL. No exemplo acima, `fn.COUNT(Livro.id).alias('livro_count')` seria traduzido para `COUNT(livro.id) AS livro_count`.

Agora vamos listar todos os autores e os titulos de todos os seus livros. Como você deve ter adivinhado, isso poderia facilmente se transformar em outra situação [N + 1](https://docs.peewee-orm.com/en/latest/peewee/relationships.html#nplusone) se não tomarmos cuidado.

Antes de mergulhar no código, considere como este exemplo é diferente do exemplo anterior, onde listamos todos os livros e o nome de seus autores. Um livro só pode ter um autor, portanto, quando realizamos a junção de livro para autor, sempre haverá uma única combinação. A situação é diferente quando estamos unindo de autor para livro porque um autor pode ter zero livros ou pode ter vários livros. Como estamos usando bancos de dados relacionais, se fôssemos fazer uma junção de autor a livro, todas as pessoas com vários livros seriam repetidas, uma para cada livro.

Ficaria assim:

In [86]:
query = (Autor
         .select(Autor, Livro)
         .join(Livro, JOIN.LEFT_OUTER)
         .order_by(Autor.nome, Livro.titulo))

for autor in query:
    if hasattr(autor, 'livro'):
        print(autor.nome, autor.livro.titulo)
    else:
        print(autor.nome, 'nenhum livro')

Aldous Leonard Huxley Brave New World
Aldous Leonard Huxley Heaven and Hell
Aldous Leonard Huxley The Doors of Perception
Clarice Lispector The Hour of the Star
Gabriel García Márquez nenhum livro
George Orwell nenhum livro
J. R. R. Tolkien The Hobbit
J. R. R. Tolkien The Lord of the Rings
Machado de Assis Dom Casmurro
Machado de Assis Memórias Póstumas de Brás Cubas
Machado de Assis Quincas Borba
Monteiro Lobato nenhum livro
Philip K. Dick Do Androids Dream of Electric Sheep?
Philip K. Dick The Cosmic Puppets
Philip K. Dick The World She Wanted
William Gibson nenhum livro


Normalmente, esse tipo de duplicação é indesejável. Para acomodar o fluxo de trabalho mais comum (e intuitivo) de listar um autor e anexar uma lista dos livros desse autor, podemos usar um método especial chamado **prefetch()**:

In [87]:
query = Autor.select().order_by(Autor.nome).prefetch(Livro)

for autor in query:
    print(autor.nome)
    for livro in autor.livros:
        print('  *', livro.titulo)

Aldous Leonard Huxley
  * Brave New World
  * The Doors of Perception
  * Heaven and Hell
Clarice Lispector
  * The Hour of the Star
Gabriel García Márquez
George Orwell
J. R. R. Tolkien
  * The Lord of the Rings
  * The Hobbit
Machado de Assis
  * Memórias Póstumas de Brás Cubas
  * Quincas Borba
  * Dom Casmurro
Monteiro Lobato
Philip K. Dick
  * The Cosmic Puppets
  * The World She Wanted
  * Do Androids Dream of Electric Sheep?
William Gibson


#### Funções SQL

Uma última consulta. Isso usará uma função SQL para encontrar todas os autores cujos nomes começam com um G maiúsculo ou minúsculo:

In [88]:
expression = fn.Lower(fn.Substr(Autor.nome, 1, 1)) == 'g'

for autor in Autor.select().where(expression):
    print(autor.nome)

George Orwell
Gabriel García Márquez


Este é apenas o básico! Você pode tornar suas consultas tão complexas quanto desejar. Verifique a documentação sobre [Querying](https://docs.peewee-orm.com/en/latest/peewee/querying.html#querying) para obter mais informações.

#### Banco de Dados

Terminamos com nosso banco de dados, vamos então fechar a conexão:

In [89]:
db.close()

True

Em um aplicativo real, existem alguns padrões estabelecidos de como você gerencia o tempo de vida da sua conexão com o banco de dados. Por exemplo, um aplicativo da web normalmente abre uma conexão no início da solicitação e fecha a conexão após gerar a resposta. Um pool de conexão pode ajudar a eliminar a latência associada aos custos de inicialização.

Para aprender a configurar seu banco de dados, consulte a [documentação do banco de dados](https://docs.peewee-orm.com/en/latest/peewee/database.html#database), que fornece muitos exemplos. O Peewee também suporta a [configuração do banco de dados em tempo de execução](https://docs.peewee-orm.com/en/latest/peewee/database.html#deferring-initialization), bem como a configuração ou alteração do banco de dados a qualquer momento.